In [ ]:
plt_aon = figure(plot_width=800, plot_height=400)
plt_aon.line(range(np.size(both_aon_64_mAvg[both_aon_64_mAvg.keys()[0]])), both_aon_64_mAvg[both_aon_64_mAvg.keys()[3]], line_width=1)

show(plt_aon)

In [ ]:
for i in range(10):
    plt.figure(figsize=(100,15))
    plt.plot(both_aon_64_mAvg[both_aon_64_mAvg.keys()[i]])
# plt.figure(figsize=(100,15))
# plt.plot(both_aon_64_mAvg[both_aon_64_mAvg.keys()[4]])

In [ ]:
for a in aon.keys(): print aon.keys().index(a), a, np.size(aon[a])

In [ ]:
# np.shape(aon[aon.keys()[0][11]])
# np.shape([[1,2,3],[4,6,7]])
print np.shape(aon[aon.keys()[0]][:0])

In [ ]:
# load data
subjectName = "Aon"
loadPath = os.path.join(dataPath, processedFolderName, subjectName) # path to load data

# for x in [i for i in os.listdir(loadPath) if not i.startswith(".")]:
#     print x
    
x = [i for i in os.listdir(loadPath) if not i.startswith(".")]
# print x

def getheader(path):
    print "getting header for", subjectName + ".."
    header = cPickle.load(open((os.path.join(path, "header.p")), "rb"))
    header = header[1:len(header)]
    return header
    
def getdata(filename):
    print "getting", filename, "data..."
    return cPickle.load(open((os.path.join(loadPath, filename)), "rb"))
        
header = getheader(loadPath)
temp = getdata(x[0])
# aon = {i:getdata(i) for i in x}


In [ ]:
print np.shape(temp)[1]
both_aon_64_mAvg = {i:temp[:,header.index(i)] for i in header}

In [ ]:
np.size(both_aon_64_mAvg[both_aon_64_mAvg.keys()[0]])

In [ ]:
# check if there is processed data
# checkProcessedIfExist()
# process and save all data
# processAll(savePath)

In [ ]:
%matplotlib inline
# ----------
# run this cell and all below first
# ----------
import os
import time
import datetime
import sys
import matplotlib.pyplot as plt
import plottingdata
import numpy as np
import cPickle
import linecache
import seaborn as sns
sns.set(style='ticks', palette='Set2')
sns.despine()


In [ ]:
# setting variables
dataPath = "/Users/tulakan/OneDrive/BME/Rehabilitation Project/EMG data/EMG Gyro 22-23 Jul 2015"  # raw data dir
processedFolderName = "Processed"
savePath = os.path.join(dataPath, processedFolderName)
printSubjtName = False
printDataFileNameFor = "no"  # name of subject to print
windowSize = 64

In [ ]:
# loop through directory -> get folder name for each subject and raw data file names
allSubject = [x for x in os.listdir(dataPath) if not x.startswith(".") and x != processedFolderName]
if printSubjtName: print allSubject

# data file names for each subject is collected as dictionary
dataFileName = {x: [i for i in os.listdir(os.path.join(dataPath, x)) if not i.startswith(".") and i.endswith(".ASC")]  for x in allSubject}
if printDataFileNameFor != "no":
    try:
        print dataFileName[printDataFileNameFor]
    except KeyError:
        print "No data for", printDataFileNameFor

In [ ]:
# functions to call
def checkProcessedIfExist():
    if not os.path.exists(os.path.join(dataPath, "Processed")):
        os.makedirs(os.path.join(dataPath, "Processed"))
    for x in allSubject:
        if not os.path.exists(os.path.join(dataPath, "Processed", x)):
            os.makedirs(os.path.join(dataPath, "Processed", x))
    for x in allSubject:
        # .p is pickle files for processed data
        # this snippet trying to find .p in processed folder
        # if found it shows files name
        # else print no .p file in current dir
        if len([i for i in os.listdir(os.path.join(dataPath, processedFolderName, x)) if i.endswith(".p")]) == 0:
            print "no .p file in", os.path.join(dataPath, processedFolderName, x)
        else:
            print x, ":", [i for i in os.listdir(os.path.join(dataPath, processedFolderName, x)) if i.endswith(".p")]
        # for y in dataFileName[x]:
        # if not os.path.exists(os.path.join(dataPath, processedFolderName, x, y.split(".")[0]+"_processed.p")):
        #     print y.split(".")[0]+"_processed.p", "does not exist.."
        # print os.path.join(dataPath, allSubject[0], dataFileName[allSubject[0]][0])

# process data
def processAll(savePath):
    # count time
    startTime = datetime.datetime.now()

    for i in allSubject:
        savedir = os.path.join(savePath, i)
        print "#########################"
        print "processing", i + "..."
        print "#########################"
        for j in dataFileName[i]:
            saveProcessed(os.path.join(dataPath, i, j), savedir, windowSize)
            print "#########################"
            print j, "process - DONE"
            print "#########################"

    # savedir = os.path.join(savePath, allSubject[0])
    # processData.saveProcessed(os.path.join(dataPath, allSubject[0], dataFileName[allSubject[0]][0]), savedir, windowSize)

    # count time
    stopTime = datetime.datetime.now()
    print "data processing takes", stopTime-startTime


def saveProcessed(filename, savedir, windowSize):
    # filename = motorSpeed+actionPlane+'.ASC'
    filenameSplit = (os.path.split(filename)[-1]).split(".")[0]
    # actionname = os.path.split(filename)[-1]  # screen dir out, only pick file name
    # print filenameSplit

    # get sampling rate
    print "get sampling rate.."
    spr = linecache.getline(filename, 1)
    spr = spr.split()[0]
    print "DONE"
    # print spr
    # get header
    print "get header.."
    header = linecache.getline(filename, 3)
    header = getHeader(header)
    print "DONE"
    # print header
    time.sleep(1)
    # get raw data
    print "get raw data.."
    rawdt = np.loadtxt(filename, skiprows=3)
    fstcoldt = rawdt[:, 0]  # first column data 1,2,3,...,len(data)
    fstcoldtassec = fstcoldt/int(spr)  # convert hertz to sec
    rawdt = rawdt[:, 1:]  # 10 channels of muscles
    print "DONE"

    # process data
    fstcoldtassec = fstcoldtassec[0:-1-windowSize+1]  # cutting out the window size
    print "rectify data.."
    rawrecdt = abs(rawdt)  # raw data rectification
    print "DONE"
    print "applying moving average filter.."
    mavgdt = movingAvgFilter(windowSize, rawrecdt)  # moving average filtered
    print "DONE"
    print "applying root mean square filter.."
    rmsdt = rmsFilter(windowSize, rawrecdt)  # root mean square filted
    print "DONE"
    rawrecdt = rawrecdt[0+(windowSize/2):-1-(windowSize/2)+1, :]  # make easier to plot

    # save data
    # savedir = os.path.join(subjectName, 'processed')
    # if not os.path.exists(savedir):
    #     os.makedirs(savedir)
    print "dump filtered data to pickle files.."
    cPickle.dump(header, open(os.path.join(savedir, 'header.p'), "wb"))
    cPickle.dump(fstcoldtassec, open(os.path.join(savedir, 'fstcoldtassec.p'), "wb"))
    cPickle.dump(rawrecdt, open(os.path.join(savedir, filenameSplit + "_" + str(windowSize) + "_rectify.p"), "wb"))
    print "rectify DONE"
    cPickle.dump(mavgdt, open(os.path.join(savedir, filenameSplit + "_" + str(windowSize) + "_mAvg.p"), "wb"))
    print "moving average DONE"
    cPickle.dump(rmsdt, open(os.path.join(savedir, filenameSplit + "_" + str(windowSize) + "_rms.p"), "wb"))
    print "root mean square DONE"
    # np.savetxt(savedir+'//'+subjectName+'_'+actionname+'_'+str(windowSize)+'_'+'rectify'+'.txt', rawrecdt)
    # np.savetxt(savedir+'//'+subjectName+'_'+actionname+'_'+str(windowSize)+'_'+'mAvg'+'.txt', mavgdt)
    # np.savetxt(savedir+'//'+subjectName+'_'+actionname+'_'+str(windowSize)+'_'+'rms'+'.txt', rmsdt)


# filter functions

# These filters will move window over data starting at data position of (window size/2)
# so that data loss will be divided to be half for initial and half for terminal

def movingAvgFilter(windowSize, data):
    startPoint = windowSize/2  # position to start windowing
    noOfData = np.size(data, 0)
    dataCol = np.size(data, 1)

    newData = np.zeros((noOfData-windowSize, dataCol))  # -1 to ignore data[:,1]
    for i in range(startPoint, noOfData-startPoint):
        newData[i-startPoint, :] = (sum(data[i-startPoint:i+startPoint, :]))/windowSize
    return newData


def rmsFilter(windowSize, data):
    startPoint = windowSize/2  # position to start windowing
    noOfData = np.size(data, 0)
    dataCol = np.size(data, 1)

    newData = np.zeros((noOfData-windowSize, dataCol))  # -1 to ignore data[:,1]
    for i in range(startPoint, noOfData-startPoint):
        newData[i-startPoint, :] = np.sqrt(sum(data[i-startPoint:i+startPoint, :]**2))/np.sqrt(windowSize)
    return newData


# get data
def getHeader(rawHeader):
    ##### header manipulation
    splited = rawHeader.split()  # split header
    jointNo = 3  # number of obj to join
    loopNo = len(splited)/jointNo  # number of obj after join
    finalHeader = [splited[0]]  # pre-allocation

    for i in range(loopNo):
        finalHeader.append(splited[(3*i)+1]+splited[(3*i)+2]+splited[(3*i)+3])  # rearrange header

    return finalHeader


# def getDataBack(savedir, subjectName, actionname, windowSize, filterType):
#     header = cPickle.load(open(os.path.join(savedir, 'header.p'), "rb"))
#     fstcoldtassec = cPickle.load(open(os.path.join(savedir, actionname+'_'+'fstcoldtassec.p'), "rb"))
#     data = np.loadtxt(savedir+'//'+subjectName+'_'+actionname+'_'+str(windowSize)+'_'+filterType+'.txt')

#     return header, fstcoldtassec, data